In [2]:
pip install -U matplotlib

  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.4.2
    Uninstalling matplotlib-3.4.2:
      Successfully uninstalled matplotlib-3.4.2
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\user\anaconda3\envs\dacon_nlp\python.exe -m pip install --upgrade pip' command.


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
!pip install -U pip
!pip install pandas
!pip install -U scikit-learn
!pip install -U tensorflow
!pip install -U tensorflow_hub
!pip install -U sentencepiece

  Using cached pip-22.2.2-py3-none-any.whl (2.0 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.3.1
    Uninstalling pip-21.3.1:
      Successfully uninstalled pip-21.3.1
  Rolling back uninstall of pip
  Moving to c:\users\user\appdata\roaming\python\python38\scripts\pip.exe
   from C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-uninstall-d8w7ldft\pip.exe
  Moving to c:\users\user\appdata\roaming\python\python38\scripts\pip3.8.exe
   from C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-uninstall-d8w7ldft\pip3.8.exe
  Moving to c:\users\user\appdata\roaming\python\python38\scripts\pip3.exe
   from C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-uninstall-d8w7ldft\pip3.exe
  Moving to c:\users\user\appdata\roaming\python\python38\site-packages\pip-21.3.1.dist-info\
   from C:\Users\user\AppData\Roaming\Python\Python38\site-packages\~ip-21.3.1.dist-info
  Moving to c:\users\user\appdata\roaming\python\python38\site-packages\pip\
   from C:\Users\user\AppDat

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pip\_internal\cli\base_command.py", line 164, in exc_logging_wrapper
    status = run_func(*args)
  File "C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pip\_internal\cli\req_command.py", line 205, in wrapper
    return func(self, options, args)
  File "C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pip\_internal\commands\install.py", line 404, in run
    installed = install_given_reqs(
  File "C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pip\_internal\req\__init__.py", line 73, in install_given_reqs
    requirement.install(
  File "C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pip\_internal\req\req_install.py", line 802, in install
    install_wheel(
  File "C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pip\_internal\operations\install\wheel.py", line 729, in install_wheel
    _install_wheel(
  Fil

You should consider upgrading via the 'C:\Users\user\anaconda3\envs\dacon_nlp\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\user\anaconda3\envs\dacon_nlp\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\user\anaconda3\envs\dacon_nlp\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\user\anaconda3\envs\dacon_nlp\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\user\anaconda3\envs\dacon_nlp\python.exe -m pip install --upgrade pip' command.


In [13]:
!pip install tensorflow

You should consider upgrading via the 'C:\Users\user\anaconda3\envs\dacon_nlp\python.exe -m pip install --upgrade pip' command.


In [2]:
import gc
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import sys
sys.path.append('../src')
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.backend import clear_session
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub
import tokenization
import warnings 
warnings.filterwarnings(action='ignore')

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print('No GPU detected')

1 Physical GPUs, 1 Logical GPU


In [4]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [18]:
sub_dir = Path('../submit')


trn_file = 'data/train.csv'
tst_file = 'data/test_x.csv'
sample_file = 'data/sample_submission.csv'
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [19]:
algo_name = 'bert'
max_len = 100
feature_name = f'n{max_len}'
model_name = f'{algo_name}_{feature_name}'


sub_file = sub_dir / f'{model_name}.csv'

In [44]:
bert_layer = hub.KerasLayer(module_url, trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()


In [31]:
!pip install -U bert-tensorflow==1.0.1
from bert import tokenization
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

You should consider upgrading via the 'C:\Users\user\anaconda3\envs\dacon_nlp\python.exe -m pip install --upgrade pip' command.


In [32]:
train = pd.read_csv(trn_file, index_col=0)
train.head()

,text,author
index,,
0,"He was almost choking. There was so much, so m...",3
1,"“Your sister asked for it, I suppose?”",2
2,"She was engaged one day as she walked, in per...",1
3,"The captain was in the porch, keeping himself ...",4
4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [33]:
test = pd.read_csv(tst_file, index_col=0)
test.head()

,text
index,
0,“Not at all. I think she is one of the most ch...
1,"""No,"" replied he, with sudden consciousness, ""..."
2,As the lady had stated her intention of scream...
3,“And then suddenly in the silence I heard a so...
4,His conviction remained unchanged. So far as I...


In [34]:
# https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub
def bert_encode(texts, tokenizer, max_len=max_len):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [36]:
import sys
from absl import flags
sys.argv=['preserve_unused_tokens=False']
flags.FLAGS(sys.argv)

['preserve_unused_tokens=False']

In [37]:
trn = bert_encode(train.text.values, tokenizer, max_len=max_len)
tst = bert_encode(test.text.values, tokenizer, max_len=max_len)
y = train['author'].values
print(trn[0].shape, tst[0].shape, y.shape)

(54879, 100) (19617, 100) (54879,)


In [47]:
def get_model(bert_layer, max_len=max_len):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    print(input_word_ids,input_mask,segment_ids)

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    x = Dense(256, activation='relu')(clf_output)
    out = Dense(n_class, activation='softmax')(x)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [39]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [42]:
tf.device("/device:GPU:0")

In [50]:
p_val = np.zeros((trn[0].shape[0], n_class))
p_tst = np.zeros((tst[0].shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn[0], y), 1):
    print(f'training model for CV #{i}')
    es = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=2,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
    
    clf = get_model(bert_layer, max_len=max_len)
    if i == 1:
        print(clf.summary())
        
    clf.fit([x[i_trn] for x in trn], 
            to_categorical(y[i_trn]),
            validation_data=([x[i_val] for x in trn], to_categorical(y[i_val])),
            epochs=10,
            batch_size=2,
            callbacks=[es])
    p_val[i_val, :] = clf.predict([x[i_val] for x in trn])
    p_tst += clf.predict(tst) / n_fold
    
    del clf
    clear_session()
    gc.collect()

training model for CV #1
KerasTensor(type_spec=TensorSpec(shape=(None, 100), dtype=tf.int32, name='input_word_ids'), name='input_word_ids', description="created by layer 'input_word_ids'") KerasTensor(type_spec=TensorSpec(shape=(None, 100), dtype=tf.int32, name='input_mask'), name='input_mask', description="created by layer 'input_mask'") KerasTensor(type_spec=TensorSpec(shape=(None, 100), dtype=tf.int32, name='segment_ids'), name='segment_ids', description="created by layer 'segment_ids'")


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[1024,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [ ]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p_val, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')